<a href="https://colab.research.google.com/github/caleb-eckerley/SatelliteTU2020/blob/master/DataScienceFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install wayback

In [ ]:
from wayback import WaybackClient
from urllib.request import urlopen
from bs4 import BeautifulSoup
from google.colab import files
import numpy as np
import pandas as pd
import pickle

In [ ]:
client = WaybackClient()
#client.session.reset()
response = client.search('https://en.wikipedia.org/wiki/Donald_Trump')
number_of_records = 8417 #Maybe scrape this from wayback in the future?
listUrl = list()
listContent = list()
contentKey = list()
lc = 0
print(lc)

0


In [ ]:
def getNext():
  return next(response)

In [ ]:
def isSameDay(url):
    global lc
    if lc != 0:
        print(f'Last date checked was: {lc.timestamp.day}/{lc.timestamp.month}/{lc.timestamp.year}')
        print(f'Date being checked is: {url.timestamp.day}/{url.timestamp.month}/{url.timestamp.year}')
        if lc.timestamp.year == url.timestamp.year and lc.timestamp.month == url.timestamp.month and lc.timestamp.day == url.timestamp.day:
            print("Is the same day")
            return True
        else:
            print("Not the same day")
    lc = url
    return False

In [ ]:
def getListOfUrl():
    for n in range(number_of_records):
        print(n)
        check = getNext()
        if(not isSameDay(check)):
            listUrl.append(check)

In [ ]:
getListOfUrl()
print(len(listUrl))

In [ ]:
def getListOfContent(n):
    try:
        url = listUrl[n].raw_url
        returnText = client.get_memento(url)
        content = returnText.content.decode()
        soup = BeautifulSoup(content, "html.parser")
        text = soup.find_all("p")
        pText = list()

        for i in range(len(text)):
            pText.append(text[i].get_text())
        listContent.append(pText)
        contentKey.append(listUrl[n])
        if(n%50):
            print(len(listContent))
    except:
        exit

In [ ]:
for n in range(len(listUrl)):
    getListOfContent(n)

In [ ]:
print(listContent[1])
print(len(listContent))
print(contentKey[1].timestamp.strftime("%d-%b-%y"))

In [ ]:
filename = 'savedWebContent'
outfile = open(filename, 'wb')

In [ ]:
pickle.dump(listContent,outfile)
outfile.close()

In [ ]:
filename2 = 'savedWebKeys'
outfile = open(filename2, 'wb')

In [ ]:
pickle.dump(contentKey, outfile)
outfile.close()

In [ ]:
files.download('savedWebContent')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('savedWebKeys')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#To open files
infile = open('savedWebKeys','rb')
keyList = pickle.load(infile)
infile.close()

infile = open('savedWebContent','rb')
contentList = pickle.load(infile)
infile.close()

In [ ]:
def contentCollab(myArray):
    newDataArray = list()
    for dataPt in myArray:
        collectedData = ''
        for portion in dataPt:
            collectedData = collectedData + portion
        #print(collectedData)
        newDataArray.append(collectedData)
    return newDataArray

In [ ]:
contentDictionary = dict()
for key in range(len(keyList)):
    date = keyList[key].timestamp.strftime("%d-%b-%y")
    data = contentCollab(contentList)
    contentDictionary[date] = list(data[key])

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
ps = PorterStemmer()

import copy

stop_words=set(stopwords.words("english"))
stop_words.add(',')
stop_words.add('.')
stop_words.add('[')
stop_words.add(']')
stop_words.add('``')
stop_words.add("''")
stop_words.add("'s")
stop_words.add("in")
stop_words.add("In")
stop_words.add('the')
stop_words.add('The')
stop_words.add('\n')
stop_words.add('(')
stop_words.add(')')
stop_words.add(':')
print(stop_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'ll', 'they', 'under', 'is', 'didn', 'shouldn', "that'll", 'ourselves', 'once', "haven't", 'doing', 'yours', 'just', 'haven', 'needn', 'all', 'whom', "weren't", 'where', 'myself', 'during', 'hasn', 'up', 'he', 'o', 'an', 'for', 'after', "wasn't", 'here', 'y', 'she', 'am', '``', "isn't", 'own', 'both', 'd', 'what', 'having', 'the', 'not', "shouldn't", 'isn', 'themselves', 'were', 'very', 'or', "'s", 'such', 'if', 'each', 'their', 'while', 'by', 'me', 'theirs', 'about', 'before', 'other', 'above', 'them', 'ma', 're', "mustn't", "hasn't", "should've", 's', 'because', 'at', 'off', 'i', 'further', ':', 'too', "you'll", 'his', "needn't", 'has', 'between', 'its', 'have', 'yourselves', 'himself', 'and', 'mightn', 'won', "don't", 'until', "didn't", "shan't", 'm', 'why

In [ ]:
textData = copy.deepcopy(data)

In [ ]:
def textCleaning():
    for textIndex in range(len(textData)):
        textData[textIndex] = textData[textIndex].lower()

textCleaning()

In [ ]:
def tokenizeText():
    tokenText = list()
    for textIndex in range(len(textData)):
        tokens = word_tokenize(textData[textIndex])
        tokenText.append(tokens)
    return tokenText

In [ ]:
def getAllWordCount():
    wordCountList = list()
    for textIndex in range(len(noStopText)):
        wordCountList.append(FreqDist(noStopText[textIndex]))
    return wordCountList

In [ ]:
def removeStopWords():
    stopList = list()
    for textIndex in range(len(tokenedText)):
        wordList = list()
        for word in tokenedText[textIndex]:
            if word not in stop_words:
                wordList.append(word)
        stopList.append(wordList)
    return stopList

In [ ]:
def stemWords():
    stemList = list()
    for textIndex in range(len(noStopText)):
        wordList = list()
        for word in noStopText[textIndex]:
            wordList.append(ps.stem(word))
        stemList.append(wordList)
    return stemList

In [ ]:
tokenedText = tokenizeText()

In [ ]:
noStopText = removeStopWords()

In [ ]:
stemText = stemWords()

In [ ]:
countedText = getAllWordCount()

In [ ]:
print(len(tokenedText))
print(len(contentKey))
print(noStopText[-1])
print(stemText[-1])
print(countedText[-1].most_common(5))

In [ ]:
filename = 'savedNoStopText'
outfile = open(filename, 'wb')
pickle.dump(noStopText,outfile)
outfile.close()

filename = 'savedStemText'
outfile = open(filename, 'wb')
pickle.dump(stemText,outfile)
outfile.close()

filename = 'savedCountedText'
outfile = open(filename, 'wb')
pickle.dump(countedText,outfile)
outfile.close()

In [ ]:
files.download('savedNoStopText')
files.download('savedStemText')
files.download('savedCountedText')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
myDataFrame = pd.DataFrame(columns=['word','frequency','date'])
for n in range(len(countedText)):
    singleDataFrame = pd.DataFrame(countedText[n].items(), columns=['word', 'frequency'])
    singleDataFrame['date'] = contentKey[n].timestamp.strftime("%d-%m-%Y")
    myDataFrame = pd.concat([myDataFrame,singleDataFrame])
myDataFrame

,word,frequency,date
0,donald,2,24-07-2004
1,john,1,24-07-2004
2,trump,14,24-07-2004
3,born,2,24-07-2004
4,june,1,24-07-2004
...,...,...,...
4924,977,1,18-11-2020
4925,collar,1,18-11-2020
4926,abdulaziz,1,18-11-2020
4927,saud,1,18-11-2020


In [ ]:
myDataFrame.to_csv(r'wordFrequencyTimeData.csv', index = False)

In [ ]:
files.download('wordFrequencyTimeData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>